# Capstone Project I Report: Data Wrangling

This report describes the data wrangling steps I performed to clean the data set for my capstone project one, as well as making it ready for further processing. The steps I take include: data merge, data summarize, data interpolation, outliers detection and explanation, data selection.

# Data Merge

The goal of this project is to predict goods sales based on weather data. The sales record is stored in train.csv, and weather record is stored in weather.csv. Data from key.csv indicates the corresponding relationship between the store and weather station.

Naturally, the first step for this project would be merging the information from the datasets together.

To merge datasets into one:

In [1]:
import pandas as pd
import numpy as np
import sys
import re
df_key = pd.read_csv("../input/key.csv")
df_train = pd.read_csv("../input/train.csv")
df_test = pd.read_csv("../input/test.csv")
df_weather = pd.read_csv("../input/weather.csv")

df_train['date'] = pd.to_datetime(df_train['date'])
df_weather['date'] = pd.to_datetime(df_weather['date'])

temp = pd.merge(df_train, df_key,how='left', on=['store_nbr'])
df_main_train = pd.merge(temp, df_weather, how='left', on=['station_nbr','date'])

print(df_train.shape)
print(temp.shape)
print(df_main_train.shape)
print(list(df_main_train))

(4617600, 4)
(4617600, 5)
(4617600, 23)
['date', 'store_nbr', 'item_nbr', 'units', 'station_nbr', 'tmax', 'tmin', 'tavg', 'depart', 'dewpoint', 'wetbulb', 'heat', 'cool', 'sunrise', 'sunset', 'codesum', 'snowfall', 'preciptotal', 'stnpressure', 'sealevel', 'resultspeed', 'resultdir', 'avgspeed']


The weather station number is first merged to sales record based on store number, and both time and station number are used to merge sales record and weather record.

## Data Summarize

The main idea of doing data summarize is to make sure the content data is as described:

In [2]:
print(df_main_train.info())
print(df_main_train.describe())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4617600 entries, 0 to 4617599
Data columns (total 23 columns):
date           datetime64[ns]
store_nbr      int64
item_nbr       int64
units          int64
station_nbr    int64
tmax           object
tmin           object
tavg           object
depart         object
dewpoint       object
wetbulb        object
heat           object
cool           object
sunrise        object
sunset         object
codesum        object
snowfall       object
preciptotal    object
stnpressure    object
sealevel       object
resultspeed    object
resultdir      object
avgspeed       object
dtypes: datetime64[ns](1), int64(4), object(18)
memory usage: 845.5+ MB
None
          store_nbr      item_nbr         units   station_nbr
count  4.617600e+06  4.617600e+06  4.617600e+06  4.617600e+06
mean   2.309108e+01  5.600000e+01  9.868756e-01  1.132329e+01
std    1.295281e+01  3.204164e+01  9.875798e+00  4.946884e+00
min    1.000000e+00  1.000000e+00  0.000000e+00  1.0

From above, we can see the following index appeared in the final dataset:

* date: year-month-day format
* store_nbr: Walmart store number
* item_nbr: item number, 117 of them, each number indicates one item, we do not have further information about what precise item would that be. 
* units: number of items sold on that day
* station_nbr: weather station number
* tmax, tmin, tavg, depart, dewpoint, wetbulb: temperature max, min, average, departure from normal, average dew point, average wet bulb. in Fahrenheit
* sunrise, sunset: time of sunrise and sunset
* codesum: special code in letters indicating the weather conditions of that day, such as RA as rain, SN as snowing 
* snowfall: snow/ice on the ground in inches at 1200 UTC
* preciptotal: 24-hour snow drop in inches
* stnpressure: air pressure
* sealevel: in meters
* resultspeed: resultant wind speed, miles per hour
* resultdir: resultant wind direciton, in degrees
* avgspeed: average wind speed, miles per hour

## Weather Event Locate & Data interpolation

Highlight the data for the weather events, which is defined as rainy days with 1 inch or more rainfall, or snowy days with 2 inches or more snowfall.

For data interpolation, pandas provide a convenient function: pd.interpolate()

In [3]:
df_ordered = df_main_train.sort_values(['store_nbr','item_nbr','date']).reset_index(drop=True)

# data interpolation after sorting
df_ordered = df_ordered.convert_objects(convert_numeric=True)
df_ordered = df_ordered.interpolate()

# weather event locate
patternRA = 'RA'
patternSN = 'SN'
df_ordered['RA'], df_ordered['SN'] = df_ordered['codesum'].str.contains(patternRA), df_ordered['codesum'].str.contains(patternSN)
df_ordered['Condition'] = (df_ordered['RA'] & (df_ordered['preciptotal']>1.0)) | (df_ordered['SN'] & (df_ordered['preciptotal']>2.0))
df_ordered['WEvent'] = (pd.rolling_mean(df_ordered['Condition'],window=7,center=True) > 0)

mask = (df_ordered['WEvent'] == True)
df_severeWeather = df_ordered.loc[mask]

print(df_ordered.info())

/home/zexi/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:4: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4617600 entries, 0 to 4617599
Data columns (total 27 columns):
date           datetime64[ns]
store_nbr      int64
item_nbr       int64
units          int64
station_nbr    int64
tmax           float64
tmin           float64
tavg           float64
depart         float64
dewpoint       float64
wetbulb        float64
heat           float64
cool           float64
sunrise        float64
sunset         float64
codesum        object
snowfall       float64
preciptotal    float64
stnpressure    float64
sealevel       float64
resultspeed    float64
resultdir      float64
avgspeed       float64
RA             bool
SN             bool
Condition      bool
WEvent         bool
dtypes: bool(4), datetime64[ns](1), float64(17), int64(4), object(1)
memory usage: 827.9+ MB
None


/home/zexi/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:12: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=7,center=True).mean()


## Outlier Detection

Looking for outliers, defined by numbers 3 std away from the main:

In [4]:
df = df_ordered.loc[df_ordered.Condition == 1][['date','preciptotal']]

df.preciptotal.mean()

df_outlier = df[np.abs(df.preciptotal-df.preciptotal.mean())>(3*df.preciptotal.std())]

grouped_df = df_outlier.groupby(['preciptotal'])['date']

for key, item in grouped_df:
    print(key)

5.36
5.67
6.93
7.36


As the most important data, 7.36 inches rainfall seems to be ok?...

In [5]:
df = df_ordered.loc[df_ordered.Condition == 1][['date','tavg']]

df.tavg.mean()

df_outlier = df[np.abs(df.tavg-df.tavg.mean())>(3*df.tavg.std())]

grouped_df = df_outlier.groupby(['tavg'])['date']

for key, item in grouped_df:
    print(key)

24.0
29.0


-4 degree is the coldest case, as one lived in central Canada I envy those guys.

In [6]:
df = df_ordered.loc[df_ordered.Condition == 1][['date','avgspeed']]

df.avgspeed.mean()

df_outlier = df[np.abs(df.avgspeed-df.avgspeed.mean())>(3*df.avgspeed.std())]

grouped_df = df_outlier.groupby(['avgspeed'])['date']

for key, item in grouped_df:
    print(key)

22.2


22.2 mph does not even consider towards strong wind...

## Data Selection: VIF

Some features have strong (linear) correlations between them, and that may effect the performance of some learning methods. So it is best to identify those correlations using variance inflation factor, also known as VIF:

In [7]:
import pandas as pd
import numpy as np
from patsy import dmatrices
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

df = df_ordered

mask = (df['item_nbr'] == 5)
df = df.loc[mask]

df_temperature_related = df[['units','tmax','tmin','tavg','depart','dewpoint','wetbulb','heat','cool']]
df_temperature_related = df_temperature_related.convert_objects(convert_numeric=True).dropna()
df_temperature_related = df_temperature_related._get_numeric_data()
df_temperature_related = df_temperature_related.reset_index(drop=True)

df_temperature_related_features = df_temperature_related[['tmax','tmin','tavg','depart','dewpoint','wetbulb','heat','cool']]

features = "+".join(df_temperature_related_features.columns)
y, X = dmatrices('units ~' + features, df_temperature_related, return_type='dataframe')

vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns

vif.round(1)

/home/zexi/anaconda2/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
/home/zexi/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:13: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
/home/zexi/anaconda2/lib/python2.7/site-packages/statsmodels/regression/linear_model.py:1386: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/home/zexi/anaconda2/lib/python2.7/site-packages/statsmodels/stats/outliers_influence.py:167: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


,VIF Factor,features
0,0.000000,Intercept
1,47.100000,tmax
2,50.500000,tmin
3,inf,tavg
4,1.000000,depart
5,18.600000,dewpoint
6,40.600000,wetbulb
7,inf,heat
8,inf,cool


In [8]:
df_rainfall_related = df[['units','snowfall','preciptotal']]
df_rainfall_related = df_rainfall_related.convert_objects(convert_numeric=True).dropna()
df_rainfall_related = df_rainfall_related._get_numeric_data()
df_rainfall_related = df_rainfall_related.reset_index(drop=True)

df_rainfall_related_features = df_rainfall_related[['snowfall','preciptotal']]

features = "+".join(df_rainfall_related_features.columns)
y, X = dmatrices('units ~' + features, df_rainfall_related, return_type='dataframe')

vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns

vif.round(1)

/home/zexi/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  from ipykernel import kernelapp as app


,VIF Factor,features
0,1.1,Intercept
1,1.0,snowfall
2,1.0,preciptotal


In [9]:
df_wind_related = df[['units','stnpressure','sealevel','resultspeed','resultdir','avgspeed']]
df_wind_related = df_wind_related.convert_objects(convert_numeric=True).dropna()
df_wind_related = df_wind_related._get_numeric_data()
df_wind_related = df_wind_related.reset_index(drop=True)

df_wind_related_features = df_wind_related[['stnpressure','sealevel','resultspeed','resultdir','avgspeed']]

features = "+".join(df_wind_related_features.columns)
y, X = dmatrices('units ~' + features, df_wind_related, return_type='dataframe')

vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns

vif.round(1)

/home/zexi/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  from ipykernel import kernelapp as app


,VIF Factor,features
0,27281.0,Intercept
1,1.2,stnpressure
2,1.1,sealevel
3,5.6,resultspeed
4,1.0,resultdir
5,5.8,avgspeed


From the results above, we can see that temperature features have high correlations between one another. Therefore it is approirate to select only one feature (tavg) into feature set. The rainfall related features are chosen into the feature set. The avgspeed and resultspeed have high correlation, therefore only one of them (avgspeed) is chosen into the feature set. The other wind-related features are also chosen into the feature set.

In [ ]:
# save data for further processing, not executed in this notebook
df_severeWeather.to_csv('severeWeather.csv', sep=',')
df_ordered.to_csv('ordered.csv', sep=',')